### 🧪 Smart AI - RAG with Role-Based Prompting

This extand version of the Smart Document Helper implements **Role Prompting / Persona Simulation** to improve response quality and user experience.

**Key featurs:**

- 🎭 **Multiple Personas**: Choose from Teacher, Expert Reviewer, Legal Advisor, Technical Writer, or Friendly Assistant
- 🎯 **Role-Specific Prompts**: Each persona has tailored prompt templates for different tasks
- 📊 **Response Quality**: Responses are contextualized based on the selected role
- 🔄 **Dynamic Interaction**: Users can switch between roles for different perspectives

**Available Roles:**
- 👩‍🏫 **Teacher**: Explains concepts clearly with examples, pedagogical approach
- 🔍 **Expert Reviewer**: Critical analysis, detailed evaluation, professional tone
- ⚖️ **Legal Advisor**: Formal language, cautious interpretation, disclaimer-aware
- 💻 **Technical Writer**: Structured, precise, documentation-style responses
- 😊 **Friendly Assistant**: Conversational, helpful, approachable tone

### 🔧 Part 0: Setup and Installations

Install the required libraries for our RAG system.

In [ ]:
!pip install faiss-cpu
!pip install gradio
!pip install pypdf

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


### 📦 Part 1: Import Required Libraries

In [ ]:
import os
import re
import torch
import faiss
import numpy as np
import gradio as gr
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig
from sentence_transformers import SentenceTransformer
from pypdf import PdfReader
import warnings
from typing import Dict, List, Tuple

# Suppress specific warnings if needed
warnings.filterwarnings("ignore", category=FutureWarning)

print("Libraries imported successfully.")

Libraries imported successfully.


### ⚙️ Part 2: Configuration – Device, Model Selection, and Role Definitions

In [ ]:
# --- Configuration ---
# Select device (GPU if available, otherwise CPU)
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"Using GPU: {torch.cuda.get_device_name(0)}")
else:
    device = torch.device("cpu")
    print("Using CPU. Note: LLM inference will be significantly slower.")

# --- Model Selection ---
EMBEDDING_MODEL_NAME = 'all-MiniLM-L6-v2'
LLM_MODEL_NAME = "google/gemma-2b-it"
TARGET_LANGUAGE = "Finnish"

# --- Role Definitions ---
ROLES = {
    "Teacher": {
        "description": "Explains concepts clearly with examples, uses pedagogical approach",
        "emoji": "👩‍🏫",
        "traits": "patient, educational, uses analogies and examples"
    },
    "Expert Reviewer": {
        "description": "Provides critical analysis and detailed evaluation",
        "emoji": "🔍",
        "traits": "analytical, thorough, objective, professional"
    },
    "Legal Advisor": {
        "description": "Formal language with cautious interpretation",
        "emoji": "⚖️",
        "traits": "formal, precise, includes disclaimers, risk-aware"
    },
    "Technical Writer": {
        "description": "Structured and precise documentation-style responses",
        "emoji": "💻",
        "traits": "structured, clear, technical, uses bullet points and sections"
    },
    "Friendly Assistant": {
        "description": "Conversational and approachable tone",
        "emoji": "😊",
        "traits": "warm, helpful, conversational, encouraging"
    }
}

# --- Global Variables ---
embedder = None
text_generator = None
tokenizer = None

Using CPU. Note: LLM inference will be significantly slower.


### 🔑 Part 3: Authenticate with Hugging Face Hub

In [ ]:
!huggingface-cli login

'huggingface-cli' is not recognized as an internal or external command,
operable program or batch file.


### 🤖 Part 4: Load Models – Embedder & LLM

In [ ]:
def load_models():
    """Loads the embedding and language models."""
    global embedder, text_generator, tokenizer

    print(f"Loading embedding model: {EMBEDDING_MODEL_NAME}...")
    try:
        embedder = SentenceTransformer(EMBEDDING_MODEL_NAME, device=device)
        print("Embedding model loaded successfully.")
    except Exception as e:
        print(f"Error loading embedding model: {e}")
        raise

    print(f"Loading LLM: {LLM_MODEL_NAME}...")
    try:
        use_4bit = True 
        bnb_config = None
        if use_4bit and torch.cuda.is_available():
            try:
                bnb_config = BitsAndBytesConfig(
                    load_in_4bit=True,
                    bnb_4bit_quant_type="nf4",
                    bnb_4bit_compute_dtype=torch.bfloat16,
                    bnb_4bit_use_double_quant=False,
                )
                print("Using 4-bit quantization.")
            except Exception as e:
                print(f"Could not set up 4-bit quantization, proceeding without it: {e}")
                bnb_config = None

        tokenizer = AutoTokenizer.from_pretrained(LLM_MODEL_NAME)

        if torch.cuda.is_available() and torch.cuda.is_bf16_supported():
             model_dtype = torch.bfloat16
             print("Using bfloat16 dtype.")
        else:
             model_dtype = torch.float16
             print("Using float16 dtype.")

        model = AutoModelForCausalLM.from_pretrained(
            LLM_MODEL_NAME,
            device_map="auto",
            torch_dtype=model_dtype,
            quantization_config=bnb_config,
            trust_remote_code=True
        )
        print("LLM loaded successfully.")

        text_generator = pipeline(
            "text-generation",
            model=model,
            tokenizer=tokenizer,
            max_new_tokens=400,  # Increased for role-based responses
            temperature=0.7,
            top_p=0.9,
            do_sample=True,
            framework="pt"
        )
        print("Text generation pipeline ready.")

    except Exception as e:
        print(f"Error loading LLM or creating pipeline: {e}")
        raise

# Load models
try:
    load_models()
except Exception as e:
    print(f"Failed to load models. Error: {e}")

Loading embedding model: all-MiniLM-L6-v2...
Embedding model loaded successfully.
Loading LLM: google/gemma-2b-it...
Error loading LLM or creating pipeline: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/google/gemma-2b-it.
401 Client Error. (Request ID: Root=1-687b9589-28638f7861a198101b394294;cf68958a-4482-49e4-8ed3-f66bfc729d48)

Cannot access gated repo for url https://huggingface.co/google/gemma-2b-it/resolve/main/config.json.
Access to model google/gemma-2b-it is restricted. You must have access to it and be authenticated to access it. Please log in.
Failed to load models. Error: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/google/gemma-2b-it.
401 Client Error. (Request ID: Root=1-687b9589-28638f7861a198101b394294;cf68958a-4482-49e4-8ed3-f66bfc729d48)

Cannot access gated repo for url https://huggingface.co/google/gemma-2b-it/resolve/main/config.json.
Access to model google/gemma-2

### 📄 Part 5: Document Processing & Vector Store Creation

In [ ]:
def load_and_chunk_pdf(file_path, chunk_size=700, chunk_overlap=70):
    """
    Loads text from a PDF file, cleans it, and splits it into overlapping chunks.
    """
    if not file_path or not os.path.exists(file_path):
        print(f"Error: PDF file not found at {file_path}")
        return None
    try:
        print(f"Loading PDF: {file_path}")
        reader = PdfReader(file_path)
        text = ""
        for i, page in enumerate(reader.pages):
            page_text = page.extract_text()
            if page_text:
                text += page_text + "\n"
            else:
                print(f"Warning: Could not extract text from page {i+1}.")

        if not text:
            print("Error: No text extracted from the PDF.")
            return None

        # Basic cleaning
        text = re.sub(r'\n\s*\n', '\n', text).strip()
        text = re.sub(r'\s+', ' ', text).strip()

        # Simple sliding window chunking
        chunks = []
        start_index = 0
        while start_index < len(text):
            end_index = start_index + chunk_size
            chunks.append(text[start_index:end_index])
            start_index += chunk_size - chunk_overlap

        chunks = [chunk for chunk in chunks if len(chunk.strip()) > 50]

        print(f"Document loaded and split into {len(chunks)} chunks.")
        return chunks
    except Exception as e:
        print(f"Error loading or chunking PDF '{file_path}': {e}")
        return None

def build_vector_store(chunks, embedder_model):
    """
    Generates embeddings for text chunks and builds a FAISS index.
    """
    if not chunks or embedder_model is None:
        print("Error: No chunks or embedder model provided.")
        return None, None
    try:
        print(f"Generating embeddings for {len(chunks)} chunks...")
        embeddings = embedder_model.encode(chunks, convert_to_tensor=False, show_progress_bar=True)
        embeddings_np = np.array(embeddings).astype('float32')

        embedding_dim = embeddings_np.shape[1]
        index = faiss.IndexFlatL2(embedding_dim)
        index.add(embeddings_np)

        print(f"FAISS index created successfully with {index.ntotal} vectors.")
        return index, chunks
    except Exception as e:
        print(f"Error building vector store: {e}")
        return None, None

### 🔎 Part 6: Semantic Retrieval from Vector Store

In [ ]:
def retrieve_context(query, vector_store, embedder_model, indexed_chunks, top_k=3):
    """
    Retrieves the top_k most relevant text chunks from the vector store.
    """
    if vector_store is None or embedder_model is None or indexed_chunks is None:
        return "Error: Vector store, embedder, or chunks not initialized."
    try:
        print(f"Retrieving context for query: '{query}'")
        query_embedding = embedder_model.encode([query], convert_to_tensor=False)
        query_embedding_np = np.array(query_embedding).astype('float32')

        distances, indices = vector_store.search(query_embedding_np, top_k)
        retrieved_chunks = [indexed_chunks[i] for i in indices[0] if 0 <= i < len(indexed_chunks)]

        if not retrieved_chunks:
            print("Warning: No relevant chunks found for the query.")
            return "Could not find relevant context for this query in the document."

        context = "\n\n---\n\n".join(retrieved_chunks)
        print(f"Retrieved {len(retrieved_chunks)} chunks.")
        return context
    except Exception as e:
        print(f"Error during context retrieval: {e}")
        return f"Error retrieving context: {e}"

### 🎭 Part 7: Role-Based Prompt Templates

This is the core featured - role-specific prompts that dramatically change how the AI responds.

In [ ]:
# Role-Based Prompt Templates

def get_role_based_qa_prompt(role: str) -> str:
    """Generate role-specific QA prompt templates."""
    
    base_instruction = """Use the following context to answer the question. 
If the answer is not found in the context, state that you cannot answer based on the provided information."""
    
    role_templates = {
        "Teacher": f"""SYSTEM: You are a knowledgeable teacher who explains concepts clearly. {base_instruction}
When answering:
- Break down complex ideas into simple terms
- Use analogies and examples where helpful
- Encourage understanding with a patient, educational tone
- If the concept is difficult, provide step-by-step explanations

CONTEXT:
{{context}}

STUDENT'S QUESTION: {{query}}

TEACHER'S RESPONSE:""",
        
        "Expert Reviewer": f"""SYSTEM: You are an expert reviewer providing critical analysis. {base_instruction}
When answering:
- Provide thorough, analytical responses
- Evaluate claims critically
- Identify strengths and potential weaknesses
- Maintain objectivity and professionalism

CONTEXT:
{{context}}

QUESTION FOR REVIEW: {{query}}

EXPERT ANALYSIS:""",
        
        "Legal Advisor": f"""SYSTEM: You are a legal advisor providing formal guidance. {base_instruction}
Important: You are not providing actual legal advice, only information based on the document.
When answering:
- Use precise, formal language
- Include appropriate disclaimers
- Be cautious in interpretation
- Highlight any assumptions or limitations

CONTEXT:
{{context}}

INQUIRY: {{query}}

LEGAL INTERPRETATION (For informational purposes only):""",
        
        "Technical Writer": f"""SYSTEM: You are a technical writer creating clear documentation. {base_instruction}
When answering:
- Use structured, precise language
- Include bullet points or numbered lists where appropriate
- Define technical terms clearly
- Maintain consistency in terminology

CONTEXT:
{{context}}

TECHNICAL QUERY: {{query}}

TECHNICAL RESPONSE:""",
        
        "Friendly Assistant": f"""SYSTEM: You are a friendly, helpful assistant. {base_instruction}
When answering:
- Use a warm, conversational tone
- Be encouraging and supportive
- Make the information accessible
- Show enthusiasm for helping

CONTEXT:
{{context}}

USER'S QUESTION: {{query}}

FRIENDLY RESPONSE:"""
    }
    
    return role_templates.get(role, role_templates["Friendly Assistant"])


def get_role_based_summary_prompt(role: str) -> str:
    """Generate role-specific summarization prompt templates."""
    
    role_templates = {
        "Teacher": """SYSTEM: You are a teacher creating an educational summary. Based on the following text, provide a summary that:
- Highlights key learning points
- Organizes information for easy understanding
- Includes main concepts and their relationships
- Uses clear, educational language

CONTEXT:
{context}

EDUCATIONAL SUMMARY:""",
        
        "Expert Reviewer": """SYSTEM: You are an expert reviewer creating a critical summary. Based on the following text, provide a summary that:
- Identifies main arguments and evidence
- Evaluates the strength of claims
- Notes any gaps or limitations
- Maintains analytical objectivity

CONTEXT:
{context}

CRITICAL SUMMARY:""",
        
        "Legal Advisor": """SYSTEM: You are a legal advisor creating a formal summary. Based on the following text, provide a summary that:
- Identifies key legal or formal points
- Uses precise, formal language
- Notes important definitions or conditions
- Maintains cautious interpretation

CONTEXT:
{context}

FORMAL SUMMARY:""",
        
        "Technical Writer": """SYSTEM: You are a technical writer creating a structured summary. Based on the following text, provide a summary that:
- Uses clear section headings
- Includes key technical details
- Maintains consistent terminology
- Organizes information logically

CONTEXT:
{context}

TECHNICAL SUMMARY:""",
        
        "Friendly Assistant": """SYSTEM: You are a friendly assistant creating an accessible summary. Based on the following text, provide a summary that:
- Makes complex ideas simple
- Uses conversational language
- Highlights the most interesting points
- Keeps the reader engaged

CONTEXT:
{context}

FRIENDLY SUMMARY:"""
    }
    
    return role_templates.get(role, role_templates["Friendly Assistant"])


def get_role_based_translation_prompt(role: str, target_language: str) -> str:
    """Generate role-specific translation prompt templates."""
    
    role_templates = {
        "Teacher": f"""SYSTEM: You are a language teacher translating for educational purposes. Translate the following text into {target_language}:
- Maintain clarity for learners
- Preserve educational value
- Keep explanations simple

CONTEXT:
{{context}}

EDUCATIONAL TRANSLATION ({target_language}):""",
        
        "Expert Reviewer": f"""SYSTEM: You are an expert translator focusing on accuracy. Translate the following text into {target_language}:
- Maintain technical precision
- Preserve nuanced meanings
- Keep professional terminology

CONTEXT:
{{context}}

PROFESSIONAL TRANSLATION ({target_language}):""",
        
        "Legal Advisor": f"""SYSTEM: You are translating legal/formal content. Translate the following text into {target_language}:
- Maintain formal register
- Preserve legal precision
- Keep official terminology

CONTEXT:
{{context}}

FORMAL TRANSLATION ({target_language}):""",
        
        "Technical Writer": f"""SYSTEM: You are translating technical documentation. Translate the following text into {target_language}:
- Maintain technical accuracy
- Keep consistent terminology
- Preserve structure

CONTEXT:
{{context}}

TECHNICAL TRANSLATION ({target_language}):""",
        
        "Friendly Assistant": f"""SYSTEM: You are creating a friendly, natural translation. Translate the following text into {target_language}:
- Use natural, conversational language
- Make it sound native
- Keep the friendly tone

CONTEXT:
{{context}}

FRIENDLY TRANSLATION ({target_language}):"""
    }
    
    return role_templates.get(role, role_templates["Friendly Assistant"])

### 🧠 Part 8: Response Generation with Role Context

In [ ]:
def generate_response(query, context, task, role, generator_pipeline):
    """
    Generates a response using role-specific prompts.
    """
    if generator_pipeline is None:
        return "Error: Text generation pipeline not initialized."
    
    if not context or "Error:" in context or "Could not find relevant context" in context:
        return f"Cannot generate response because context retrieval failed: {context}"

    try:
        # Select appropriate prompt template based on task and role
        if task == "Ask a question":
            prompt_template = get_role_based_qa_prompt(role)
            prompt = prompt_template.format(context=context, query=query)
        elif task == "Summarize":
            prompt_template = get_role_based_summary_prompt(role)
            prompt = prompt_template.format(context=context)
        elif task == f"Translate (to {TARGET_LANGUAGE})":
            prompt_template = get_role_based_translation_prompt(role, TARGET_LANGUAGE)
            prompt = prompt_template.format(context=context)
        else:
            return "Error: Invalid task selected."

        print(f"Generating response with LLM using role: {role}...")
        outputs = generator_pipeline(prompt)
        generated_text = outputs[0]['generated_text']

        # Clean up the response based on role markers
        role_markers = [
            "TEACHER'S RESPONSE:", "EXPERT ANALYSIS:", "LEGAL INTERPRETATION",
            "TECHNICAL RESPONSE:", "FRIENDLY RESPONSE:", "EDUCATIONAL SUMMARY:",
            "CRITICAL SUMMARY:", "FORMAL SUMMARY:", "TECHNICAL SUMMARY:",
            "FRIENDLY SUMMARY:", f"EDUCATIONAL TRANSLATION ({TARGET_LANGUAGE}):",
            f"PROFESSIONAL TRANSLATION ({TARGET_LANGUAGE}):",
            f"FORMAL TRANSLATION ({TARGET_LANGUAGE}):",
            f"TECHNICAL TRANSLATION ({TARGET_LANGUAGE}):",
            f"FRIENDLY TRANSLATION ({TARGET_LANGUAGE}):"
        ]
        
        response = generated_text
        for marker in role_markers:
            if marker in response:
                response = response.split(marker)[-1].strip()
                break

        print("LLM Generation complete.")
        return response

    except Exception as e:
        print(f"Error during LLM generation: {e}")
        if "CUDA out of memory" in str(e):
            return "Error: GPU out of memory. Try a smaller model or enable 4-bit quantization."
        return f"Error generating response: {e}"

### 🎭 Part 9: Gradio Interface with Role Selection

In [ ]:
# Gradio Interface Logic with Role Selection

# Store vector store state
document_state = {
    "file_path": None,
    "vector_store": None,
    "indexed_chunks": None
}

def process_document_and_query(file_obj, task, query, role):
    """
    Main function with role-based processing.
    """
    global document_state

    status_message = ""
    result_output = ""

    # Check Models
    if embedder is None or text_generator is None:
        status_message = "Error: Models not loaded. Please check console."
        print(status_message)
        return status_message, result_output

    # Process Document
    current_file_path = file_obj.name if file_obj else None

    if current_file_path is None:
        status_message = "Please upload a PDF document."
        return status_message, result_output

    if current_file_path != document_state.get("file_path"):
        status_message = f"Processing new document: {os.path.basename(current_file_path)}..."
        print(status_message)
        document_state = {"file_path": None, "vector_store": None, "indexed_chunks": None}

        chunks = load_and_chunk_pdf(current_file_path)
        if chunks is None:
            status_message = "Error: Failed to load or chunk the PDF."
            print(status_message)
            return status_message, result_output

        vector_store, indexed_chunks = build_vector_store(chunks, embedder)
        if vector_store is None:
            status_message = "Error: Failed to build the vector store."
            print(status_message)
            return status_message, result_output

        document_state["file_path"] = current_file_path
        document_state["vector_store"] = vector_store
        document_state["indexed_chunks"] = indexed_chunks
        status_message = f"Document processed successfully. Using role: {ROLES[role]['emoji']} {role}"
        print(status_message)
    else:
        status_message = f"Using cached document. Role: {ROLES[role]['emoji']} {role}"
        print(status_message)
        vector_store = document_state["vector_store"]
        indexed_chunks = document_state["indexed_chunks"]

    # Perform Selected Task with Role
    print(f"Task: {task}, Role: {role}")

    # Set retrieval parameters based on task
    if task == "Ask a question":
        if not query:
            status_message += "\nPlease enter a question."
            return status_message, result_output
        context_query = query
        top_k = 3
    elif task == "Summarize":
        context_query = "Provide a comprehensive overview of the document's content."
        top_k = 6
    elif task == f"Translate (to {TARGET_LANGUAGE})":
        if not query:
            if not indexed_chunks:
                status_message += "\nError: Cannot translate without document content."
                return status_message, result_output
            context_query = indexed_chunks[0][:150] + "..."
        else:
            context_query = query
        top_k = 3
    else:
        status_message += "\nError: Invalid task selected."
        return status_message, result_output

    # Retrieve context
    context = retrieve_context(context_query, vector_store, embedder, indexed_chunks, top_k=top_k)

    # Generate response with role
    if "Error:" in context or "Could not find relevant context" in context:
        result_output = f"Failed to retrieve context: {context}"
    else:
        result_output = generate_response(query, context, task, role, text_generator)

    status_message += f"\nTask '{task}' completed with {role} persona."
    print(status_message)

    return status_message, result_output


def get_role_info(role):
    """Provide information about the selected role."""
    if role in ROLES:
        info = ROLES[role]
        return f"{info['emoji']} **{role}**: {info['description']}\n\n*Traits: {info['traits']}*"
    return "Select a role to see its description."

### 🖥️ Part 10: Create and Launch the Gradio Interface

In [ ]:
# Create and Launch the Gradio Interface

print("Setting up Gradio interface with Role-Based Prompting...")

# Custom CSS for better UI
custom_css = """
.role-info {
    background-color: #f0f0f0;
    padding: 10px;
    border-radius: 5px;
    margin: 10px 0;
}
"""

# Create the interface
with gr.Blocks(css=custom_css, title="Smart AI with Role-Based Prompting") as iface:
    gr.Markdown(
        """
        # 🎭 Smart Document Helper with Role-Based AI
        
        Upload a PDF and let our AI assist you with different personas!
        Each role provides a unique perspective and communication style.
        """
    )
    
    with gr.Row():
        with gr.Column(scale=1):
            file_input = gr.File(
                label="📄 Upload PDF Document", 
                file_types=[".pdf"]
            )
            
            role_dropdown = gr.Dropdown(
                label="🎭 Select AI Role",
                choices=list(ROLES.keys()),
                value="Friendly Assistant",
                info="Choose how the AI should respond"
            )
            
            role_info = gr.Markdown(
                value=get_role_info("Friendly Assistant"),
                elem_classes=["role-info"]
            )
            
            task_dropdown = gr.Dropdown(
                label="📋 Select Task",
                choices=["Ask a question", "Summarize", f"Translate (to {TARGET_LANGUAGE})"],
                value="Ask a question"
            )
            
            query_input = gr.Textbox(
                label="❓ Enter Question or Topic",
                placeholder="What would you like to know about the document?",
                lines=3
            )
            
            submit_btn = gr.Button("🚀 Process", variant="primary")
        
        with gr.Column(scale=2):
            status_output = gr.Textbox(
                label="📊 Status", 
                interactive=False,
                lines=2
            )
            
            result_output = gr.Textbox(
                label="💬 AI Response", 
                lines=20, 
                interactive=False
            )
    
    # Examples section
    gr.Markdown("### 📚 Example Queries")
    gr.Examples(
        examples=[
            [None, "Ask a question", "What is the main purpose of this document?", "Teacher"],
            [None, "Ask a question", "What are the key findings?", "Expert Reviewer"],
            [None, "Summarize", "", "Technical Writer"],
            [None, "Summarize", "", "Legal Advisor"],
            [None, f"Translate (to {TARGET_LANGUAGE})", "Introduction section", "Friendly Assistant"]
        ],
        inputs=[file_input, task_dropdown, query_input, role_dropdown],
        cache_examples=False
    )
    
    # Event handlers
    role_dropdown.change(
        fn=get_role_info,
        inputs=[role_dropdown],
        outputs=[role_info]
    )
    
    submit_btn.click(
        fn=process_document_and_query,
        inputs=[file_input, task_dropdown, query_input, role_dropdown],
        outputs=[status_output, result_output]
    )
    
    gr.Markdown(
        f"""
        ---
        ### 🛠️ Technical Details
        - **Embedding Model**: {EMBEDDING_MODEL_NAME}
        - **Language Model**: {LLM_MODEL_NAME}
        - **Extended**: Role-Based Prompting with 5 distinct personas
        - **Features**: Dynamic prompt adaptation based on selected role
        """
    )

print("Launching Gradio interface...")
if __name__ == "__main__":
    iface.launch(debug=False, share=True)

print("Interface with role-based prompting is ready!")

Setting up Enhanced Gradio interface with Role-Based Prompting...
Launching Enhanced Gradio interface...
* Running on local URL:  http://127.0.0.1:7860

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


Enhanced interface with role-based prompting is ready!


Error: Models not loaded. Please check console.
